In [1]:
import geopandas as gpd
import pandas as pd
import fiona
import matplotlib.pyplot as plt
from shapely.geometry import Point
from scipy.spatial import cKDTree
from tqdm import tqdm
import rasterio
import numpy as np
import yaml
import os

# Load configuration
with open('../../config.yaml', 'r') as file:
    config = yaml.safe_load(file)

def resolve_path(relative_path):
    return os.path.join(config['base_path'], relative_path)

# List of ISO3 codes for Northern African countries
northern_african_countries = ['DZA', 'EGY', 'LBY', 'MAR', 'SDN', 'TUN', 'ESH', 'MRT']

#### Prep the groundwater productivity data

In [2]:
# Load the groundwater productivity data
df_GW = pd.read_csv(resolve_path(config['Groundwater_Productivity_path']), sep=r'\s+')

# Create geometry for groundwater productivity points
geometry = [Point(xy) for xy in tqdm(zip(df_GW['X'], df_GW['Y']), desc="Creating geometries")]
gdf_GW = gpd.GeoDataFrame(df_GW, geometry=geometry)
gdf_GW.set_crs(epsg=4326, inplace=True)
gdf_GW.to_file(resolve_path(config['CPIS_Groundwater_Prod_gpkg_path']))
gdf_GW = gdf_GW.to_crs(epsg=3857)

# Map groundwater productivity to categories
value_map_cat = {
    'VH': '>20',
    'H': '5-20',
    'M': '1-5',
    'LM': '0.5-1',
    'L': '0.1-0.5',
    'VL': '<0.1'
}
gdf_GW['GW_prod_cat_L_s'] = gdf_GW['GWPROD_V2'].map(value_map_cat)


Creating geometries: 1019265it [00:06, 156888.74it/s]


In [3]:
print(gdf_GW.head())

         X          Y GWPROD_V2                         geometry  \
0  9.41405  37.251192         M  POINT (1047967.285 4474177.758)   
1  9.46405  37.251192         M  POINT (1053533.259 4474177.758)   
2  9.51405  37.251192         M  POINT (1059099.234 4474177.758)   
3  9.56405  37.251192         M  POINT (1064665.209 4474177.758)   
4  9.61405  37.251192         H  POINT (1070231.183 4474177.758)   

  GW_prod_cat_L_s  
0             1-5  
1             1-5  
2             1-5  
3             1-5  
4            5-20  


Assign each Irrgated Pixel a GW Value and Produce Clean Dataset

In [4]:

# Load the AEI data by country
AEI_by_country = gpd.read_file(resolve_path(config['AEI_2015_by_Country_All_shp_path']))
AEI_by_country = AEI_by_country.to_crs(gdf_GW.crs)

# Perform a spatial join to combine AEI with groundwater productivity
gdf_combined = gpd.sjoin(gdf_GW, AEI_by_country, how="left", predicate="within")
gdf_combined = gdf_combined.rename(columns={'GWPROD_V2': 'GW_prod_cat'})
gdf_combined = gdf_combined.drop(columns=['index_right']).reset_index(drop=True)

#print(gdf_combined.head())

# Group by country and groundwater productivity category to calculate total AEI
AEI_by_country_and_prod = gdf_combined.groupby(['ISO3', 'GW_prod_cat'])['raster_val'].sum().reset_index()

# Calculate total AEI per country
total_AEI_per_country = gdf_combined.groupby('ISO3')['raster_val'].sum().reset_index()
total_AEI_per_country = total_AEI_per_country.rename(columns={'raster_val': 'Total_AEI'})

# Merge the AEI per category with the total AEI per country
merged_with_AEI = AEI_by_country_and_prod.merge(total_AEI_per_country, on='ISO3')

# Calculate the percentage of AEI per groundwater productivity category within each country
merged_with_AEI['Percent_AEI'] = (merged_with_AEI['raster_val'] / merged_with_AEI['Total_AEI'])

# Restrict to SSA
merged = merged_with_AEI[~merged_with_AEI['ISO3'].isin(northern_african_countries)]

print(merged.head())

  ISO3 GW_prod_cat    raster_val      Total_AEI  Percent_AEI
0  AGO           H  55144.969337  168115.312119     0.328019
1  AGO           L   2917.207626  168115.312119     0.017352
2  AGO          LM  16349.098923  168115.312119     0.097249
3  AGO           M  93704.036232  168115.312119     0.557380
4  AGO          VH      0.000000  168115.312119     0.000000


In [5]:
print(AEI_by_country.head())
print(gdf_GW.head())

    raster_val ISO3                                           geometry
0   388.435089  TUN  POLYGON ((1038981.914 4462363.503, 1029705.290...
1   207.121201  TUN  POLYGON ((1057535.163 4474011.088, 1057535.163...
2   721.080872  TUN  POLYGON ((1066811.787 4474011.088, 1066811.787...
3   962.729492  TUN  POLYGON ((1076088.411 4474011.088, 1076088.411...
4  2777.154541  TUN  POLYGON ((1103918.284 4474011.088, 1113194.908...
         X          Y GWPROD_V2                         geometry  \
0  9.41405  37.251192         M  POINT (1047967.285 4474177.758)   
1  9.46405  37.251192         M  POINT (1053533.259 4474177.758)   
2  9.51405  37.251192         M  POINT (1059099.234 4474177.758)   
3  9.56405  37.251192         M  POINT (1064665.209 4474177.758)   
4  9.61405  37.251192         H  POINT (1070231.183 4474177.758)   

  GW_prod_cat_L_s  
0             1-5  
1             1-5  
2             1-5  
3             1-5  
4            5-20  


In [6]:
# Load the CPIS shapefile
gdf_CPIS = gpd.read_file(resolve_path(config['Combined_CPIS_All_by_Country_shp_path']))
gdf_CPIS = gdf_CPIS.to_crs(gdf_GW.crs)

# Calculate centroids of the CPIS polygons
gdf_CPIS['centroid'] = gdf_CPIS.geometry.centroid

# Convert centroids to coordinates for KDTree
cp_centroids_coords = np.array(list(gdf_CPIS['centroid'].apply(lambda geom: (geom.x, geom.y))))

# Extract coordinates of groundwater points
gw_coords = np.array(list(gdf_GW.geometry.apply(lambda geom: (geom.x, geom.y))))

# Build a KDTree for the groundwater points
tree = cKDTree(gw_coords)

# Query the nearest groundwater point for each CPIS centroid
_, idx = tree.query(cp_centroids_coords, k=1)

# Assign the nearest groundwater productivity value to each CPIS
gdf_CPIS['GW_prod_cat'] = gdf_GW.iloc[idx]['GWPROD_V2'].values

# Remove the centroid column afterwards
gdf_CPIS = gdf_CPIS.drop(columns=['centroid'])

# Check
print(gdf_CPIS[['ISO3', 'GW_prod_cat', 'geometry']].head())

# Restrict to SSA
gdf_CPIS_SSA = gdf_CPIS[~gdf_CPIS['ISO3'].isin(northern_african_countries)]
print(f'gdf_CPIS_SSA: {gdf_CPIS_SSA.head()}')

# Group by groundwater productivity category to calculate the total number of CPIS (per level overall and by country)
CPIS_SSA_per_country_level = gdf_CPIS_SSA.groupby(['ISO3', 'GW_prod_cat']).size().reset_index(name='CPIS_per_country_level')
print(CPIS_SSA_per_country_level.head())

CPIS_SSA_per_level = gdf_CPIS_SSA.groupby('GW_prod_cat').size().reset_index(name='CPIS_count')
print(CPIS_SSA_per_level.head())

  ISO3 GW_prod_cat                                           geometry
0  ZAF           L  POLYGON ((2158598.021 -4057555.506, 2158658.42...
1  ZAF           L  POLYGON ((2158740.433 -4056950.848, 2158788.75...
2  ZAF           L  POLYGON ((2286656.768 -4053677.724, 2286693.00...
3  ZAF           L  POLYGON ((2285644.300 -4053175.531, 2285704.69...
4  ZAF           L  POLYGON ((2184297.066 -4052314.792, 2184333.29...
gdf_CPIS_SSA:   ISO3                                           geometry GW_prod_cat
0  ZAF  POLYGON ((2158598.021 -4057555.506, 2158658.42...           L
1  ZAF  POLYGON ((2158740.433 -4056950.848, 2158788.75...           L
2  ZAF  POLYGON ((2286656.768 -4053677.724, 2286693.00...           L
3  ZAF  POLYGON ((2285644.300 -4053175.531, 2285704.69...           L
4  ZAF  POLYGON ((2184297.066 -4052314.792, 2184333.29...           L
  ISO3 GW_prod_cat  CPIS_per_country_level
0  AGO           H                      21
1  AGO           L                       2
2  AGO          L

In [7]:
# OG VERSION
# Load the CPIS shapefile
#gdf_CPIS = gpd.read_file(resolve_path(config['Combined_CPIS_All_by_Country_shp_path']))
#gdf_CPIS = gdf_CPIS.to_crs(gdf_GW.crs)

# Calculate centroids of the CPIS polygons
#gdf_CPIS['centroid'] = gdf_CPIS.geometry.centroid
#print(gdf_CPIS.head())

# Create a new GeoDataFrame using the centroids
#gdf_CPIS_centroids = gpd.GeoDataFrame(gdf_CPIS.drop(columns='geometry'), geometry=gdf_CPIS['centroid'])

# Convert geometries to arrays of coordinates
#cp_centroids_coords = np.array(list(gdf_CPIS_centroids.geometry.apply(lambda geom: (geom.x, geom.y))))
#print(cp_centroids_coords)
#gw_coords = np.array(list(gdf_GW.geometry.apply(lambda geom: (geom.x, geom.y))))
#print(gw_coords)

# Build a KDTree for efficient nearest-neighbor lookup
#tree = cKDTree(gw_coords)

# Query the nearest groundwater productivity point for each CPIS centroid
#_, idx = tree.query(cp_centroids_coords, k=1)

#print(gdf_CPIS_centroids.head())

# Assign the nearest groundwater productivity value to each CPIS
#gdf_CPIS_centroids['GW_prod_cat'] = gdf_GW.iloc[idx]['GWPROD_V2'].values

# Drop rows where no groundwater productivity data was found (if any)
#gdf_CPIS_centroids = gdf_CPIS_centroids.dropna(subset=['GW_prod_cat'])

# Restrict to SSA
#gdf_CPIS_centroids = gdf_CPIS_centroids[~gdf_CPIS_centroids['ISO3'].isin(northern_african_countries)]
#print(gdf_CPIS_centroids.head())

# Group by groundwater productivity category to calculate the total number of CPIS (per level overall and by country)
#cpis_per_level = gdf_CPIS_centroids.groupby('GW_prod_cat')['GW_prod_cat'].count().reset_index(name='CPIS_count')

#print(gdf_CPIS_centroids.groupby(['ISO3', 'GW_prod_cat']).head())
#cpis_count_per_country_level = gdf_CPIS_centroids.groupby(['ISO3', 'GW_prod_cat']).size().reset_index(name='CPIS_count_per_country_level')



In [8]:
# HOPEFULLY USELESS - points_per_country
# Group by 'ISO3' and count the number of points in each country
#points_per_country = gdf_CPIS_centroids.groupby('ISO3').size().reset_index(name='Point_Count')

# Inspect the resulting DataFrame
#print(points_per_country)

In [9]:
# Perform the merge on 'ISO3' and 'GW_prod_cat'
merged_with_cpis = merged_with_AEI.merge(CPIS_SSA_per_country_level, 
                                on=['ISO3', 'GW_prod_cat'], 
                                how='left')

# Print the number of NaN values
print(f'Number of NaN values in CPIS_per_country_level before replacement: {merged_with_cpis['CPIS_per_country_level'].isna().sum()}')

# Replace NaN values in the 'CPIS_count_per_country_level' column with 0
merged_with_cpis['CPIS_per_country_level'] = merged_with_cpis['CPIS_per_country_level'].fillna(0)

# Convert the column to an integer type if needed
merged_with_cpis['CPIS_per_country_level'] = merged_with_cpis['CPIS_per_country_level'].astype(int)

# Add back the corresponding category values
merged_with_cpis['GW_prod_cat_L_s'] = merged_with_cpis['GW_prod_cat'].map(value_map_cat)


print(f'Merged w cpis: {merged_with_cpis.tail()}')
print(f'Merged: {merged_with_AEI.tail()}')
print(merged_with_cpis.tail())
print(merged_with_cpis['ISO3'].unique())


Number of NaN values in CPIS_per_country_level before replacement: 209
Merged w cpis:     ISO3 GW_prod_cat     raster_val      Total_AEI  Percent_AEI  \
253  ZWE           L  251910.184516  478508.667487     0.526449   
254  ZWE          LM   90636.649854  478508.667487     0.189415   
255  ZWE           M   92136.548785  478508.667487     0.192549   
256  ZWE          VH     482.627903  478508.667487     0.001009   
257  ZWE          VL       0.000000  478508.667487     0.000000   

     CPIS_per_country_level GW_prod_cat_L_s  
253                    1702         0.1-0.5  
254                      38           0.5-1  
255                      55             1-5  
256                       0             >20  
257                       0            <0.1  
Merged:     ISO3 GW_prod_cat     raster_val      Total_AEI  Percent_AEI
253  ZWE           L  251910.184516  478508.667487     0.526449
254  ZWE          LM   90636.649854  478508.667487     0.189415
255  ZWE           M   92136.548785

In [10]:

# Final Calculation of Targeting Ratio
targeting_ratios = []

for prod_level in CPIS_SSA_per_level['GW_prod_cat']:
    # Total CPIS count for this productivity level
    CPIS_count = CPIS_SSA_per_level[CPIS_SSA_per_level['GW_prod_cat'] == prod_level]['CPIS_count'].values[0]
    print(f'CPIS count for {prod_level}: {CPIS_count}')

    # Filter data for this productivity level
    country_data = merged_with_cpis[merged_with_cpis['GW_prod_cat'] == prod_level]
    
    # Calculate the denominator: sum over countries [(CPIS count in country) * (Percent_AEI)]
    denominator = 0
    for country in country_data['ISO3'].unique():
        # Get CPIS count per level per country
        
        country_cpis_count = merged_with_cpis[
            (merged_with_cpis['ISO3'] == country) & 
            (merged_with_cpis['GW_prod_cat'] == prod_level)
        ]['CPIS_per_country_level'].values[0]
        
        # Check:
        print(f'country_cpis_count for {prod_level} in {country}: {country_cpis_count}')

        # Get Percent_AEI for this country and productivity level
        percent_aei = merged_with_cpis[merged_with_cpis['ISO3'] == country]['Percent_AEI'].values[0]
        percent_aei = merged_with_cpis[
            (merged_with_cpis['ISO3'] == country) & 
            (merged_with_cpis['GW_prod_cat'] == prod_level)
        ]['Percent_AEI'].values[0]
        
        # Check:
        print(f'percent_aei for {prod_level} in {country}: {percent_aei}')

        # Skip if either value is nan
        if np.isnan(country_cpis_count) or np.isnan(percent_aei):
            print(f'Problem with {prod_level} in {country}')
            continue

        # Add to the denominator
        print(f'{denominator} + {country_cpis_count} * {percent_aei} =')
        denominator = denominator + country_cpis_count * percent_aei

        
    print(f'Denom for {prod_level}: {denominator}')

    
    # Calculate the targeting ratio
    if denominator == 0:
        print(f'Why is the denom for {prod_level} 0 mannn')
        targeting_ratio = np.nan  # Handle zero denominator
    else:
        targeting_ratio = CPIS_count / denominator
    
    # Store the result
    targeting_ratios.append({'GW_prod_cat_L_s': prod_level, 'Targeting_Ratio': targeting_ratio})

# Convert targeting ratios to DataFrame for easy viewing
df_targeting_ratios = pd.DataFrame(targeting_ratios)

# Display the resulting targeting ratios
print(df_targeting_ratios)


CPIS count for H: 1546
country_cpis_count for H in AGO: 21
percent_aei for H in AGO: 0.328018719071724
0 + 21 * 0.328018719071724 =
country_cpis_count for H in BDI: 0
percent_aei for H in BDI: 0.0
6.888393100506203 + 0 * 0.0 =
country_cpis_count for H in BEN: 0
percent_aei for H in BEN: 0.6663775346497173
6.888393100506203 + 0 * 0.6663775346497173 =
country_cpis_count for H in BFA: 0
percent_aei for H in BFA: 0.04605914363575515
6.888393100506203 + 0 * 0.04605914363575515 =
country_cpis_count for H in BWA: 7
percent_aei for H in BWA: 0.0002263692260203664
6.888393100506203 + 7 * 0.0002263692260203664 =
country_cpis_count for H in CAF: 0
percent_aei for H in CAF: nan
Problem with H in CAF
country_cpis_count for H in CIV: 0
percent_aei for H in CIV: 0.0
6.889977685088346 + 0 * 0.0 =
country_cpis_count for H in CMR: 0
percent_aei for H in CMR: 0.8521800168434079
6.889977685088346 + 0 * 0.8521800168434079 =
country_cpis_count for H in COD: 0
percent_aei for H in COD: nan
Problem with H in 